## 데이터 Augmentation
- 데이터가 적은 이미지에서 최대한 많은 정보를 뽑아내서 학습하는 기법
- 이미지를 사용할 때마다 임의로 변형을 가하여 더 많은 이미지를 학습하는 것과 같은 효과를 냄
- **과적합(overfitting) 방지**
- ImageDataGenerator 클래스
    - 학습 도중에 이미지에 임의 변형 및 정규화 적용
    - generator 생성
        - 변형된 이미지를 배치 단위로 불러옴
        - flow(data, labels)
        - flow_from_directory(directory) 두 가지 함수를 사용
    - fir_generator, evaluate_generator 함수를 이용하여 generator로 이미지를 불러와서 모델을 학습

In [1]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from PIL import Image
import numpy as np

datagen = ImageDataGenerator(rotation_range=40, #이미지 회전 범위 (degrees)
                            width_shift_range=0.2, # 수평, 수직 랜덤 평행이동
                            height_shift_range=0.2,
                            rescale=1./255, # 입력값을 모델을 효과적으로 학습시키기에 너무 높은 값 255로 나눔
                            shear_range = 0.2, #임의 전단 변환 (shearing transformation) 범위
                            zoom_range = 0.2, #임의 확대/축소 범위
                            horizontal_flip=True, # True로 설정할 경우, 50% 확률로 이미지를 수평으로 뒤집음. 
                             fill_mode='nearest' #이미지를 회전, 이동하거나 축소할 때 생기는 공간을 채우는 방식
                            )

# image 읽어오기
pil_img = Image.open('data/cat/cat1.jpg')
np_img = np.array(pil_img.convert('RGB'))
np_img = np_img[np.newaxis, :]
print(np_img.shape)

# case 1 - datagen.flow() 함수
i=0;
for batch in datagen.flow(np_img, batch_size=1,
                          save_to_dir='data', save_prefix='aug_cat', save_format='jpg'):
    i+=1
    if i>20:
        print('20장 생성완료')
        break

(1, 112, 112, 3)
